In [1]:
# Import the required libraries
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
from finta import TA
from pandas.tseries.offsets import DateOffset
import os
import requests
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout

%matplotlib inline

In [2]:
# Import indicator dataframe
# Start with 1-minute... alghough we would like to attatch them all together soon.
df = pd.read_csv("../algotrader2/resources/aapl_1min_pivot_point_indicator_df.csv")
df.head()

# df = pd.read_csv("../algotrader2/resources/aapl_3min_pivot_point_indicator_df.csv")
# df.head()

# df = pd.read_csv("../algotrader2/resources/aapl_1min_pivot_point_indicator_df.csv")
# df.head()



,timestamp,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,...,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC,signal,new_signal
0,2023-01-03 15:04:00+00:00,126.5001,126.880,126.4166,5733,126.820,513722,126.624892,-0.002516,126.87386,...,-65.583650,-4.458619,False,-9.952525e+06,-43.838967,-25.409400,1.034633,0.0,0,1
1,2023-01-03 15:05:00+00:00,126.8500,126.895,126.4400,3763,126.505,362900,126.625878,0.002766,126.80786,...,-40.134546,-4.465828,False,-9.402193e+06,-43.321203,-24.707011,0.967187,0.0,0,1
2,2023-01-03 15:06:00+00:00,126.5800,126.870,126.5300,2930,126.860,240388,126.664497,-0.002128,126.71786,...,-47.104800,-4.057573,False,-9.798127e+06,-39.135081,-24.923630,0.907692,0.0,0,1
3,2023-01-03 15:07:00+00:00,126.7100,126.870,126.5000,3189,126.580,289721,126.666956,0.001027,126.69186,...,-38.479113,-3.693347,False,-9.594539e+06,-32.305140,-24.862300,0.792548,0.0,0,1
4,2023-01-03 15:08:00+00:00,126.7066,126.760,126.5601,2861,126.705,255144,126.687038,-0.000027,126.66934,...,-38.583764,-3.567436,False,-9.590454e+06,-32.682043,-24.646221,0.705603,0.0,0,1


In [3]:
# our df does not have date-time
hello = df['timestamp']
hello

0         2023-01-03 15:04:00+00:00
1         2023-01-03 15:05:00+00:00
2         2023-01-03 15:06:00+00:00
3         2023-01-03 15:07:00+00:00
4         2023-01-03 15:08:00+00:00
                    ...            
153677    2024-01-17 00:47:00+00:00
153678    2024-01-17 00:52:00+00:00
153679    2024-01-17 00:55:00+00:00
153680    2024-01-17 00:57:00+00:00
153681    2024-01-17 00:58:00+00:00
Name: timestamp, Length: 153682, dtype: object

In [4]:
# Create our timestamp column as a datetime index, then save it as our index
df['timestamp'] = pd.to_datetime(df['timestamp'])

hello = df['timestamp']
hello

0        2023-01-03 15:04:00+00:00
1        2023-01-03 15:05:00+00:00
2        2023-01-03 15:06:00+00:00
3        2023-01-03 15:07:00+00:00
4        2023-01-03 15:08:00+00:00
                    ...           
153677   2024-01-17 00:47:00+00:00
153678   2024-01-17 00:52:00+00:00
153679   2024-01-17 00:55:00+00:00
153680   2024-01-17 00:57:00+00:00
153681   2024-01-17 00:58:00+00:00
Name: timestamp, Length: 153682, dtype: datetime64[ns, UTC]

In [5]:
df.set_index('timestamp', inplace=True)

df.head()

,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,SMA10,...,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC,signal,new_signal
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-01-03 15:04:00+00:00,126.5001,126.880,126.4166,5733,126.820,513722,126.624892,-0.002516,126.87386,127.15340,...,-65.583650,-4.458619,False,-9.952525e+06,-43.838967,-25.409400,1.034633,0.0,0,1
2023-01-03 15:05:00+00:00,126.8500,126.895,126.4400,3763,126.505,362900,126.625878,0.002766,126.80786,127.06139,...,-40.134546,-4.465828,False,-9.402193e+06,-43.321203,-24.707011,0.967187,0.0,0,1
2023-01-03 15:06:00+00:00,126.5800,126.870,126.5300,2930,126.860,240388,126.664497,-0.002128,126.71786,126.94739,...,-47.104800,-4.057573,False,-9.798127e+06,-39.135081,-24.923630,0.907692,0.0,0,1
2023-01-03 15:07:00+00:00,126.7100,126.870,126.5000,3189,126.580,289721,126.666956,0.001027,126.69186,126.87129,...,-38.479113,-3.693347,False,-9.594539e+06,-32.305140,-24.862300,0.792548,0.0,0,1
2023-01-03 15:08:00+00:00,126.7066,126.760,126.5601,2861,126.705,255144,126.687038,-0.000027,126.66934,126.81495,...,-38.583764,-3.567436,False,-9.590454e+06,-32.682043,-24.646221,0.705603,0.0,0,1


In [6]:
# X is everything except the new_signal column
X = df.drop('new_signal', axis=1)

X.head()

,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,SMA10,...,COPP,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC,signal
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-01-03 15:04:00+00:00,126.5001,126.880,126.4166,5733,126.820,513722,126.624892,-0.002516,126.87386,127.15340,...,-2.878051,-65.583650,-4.458619,False,-9.952525e+06,-43.838967,-25.409400,1.034633,0.0,0
2023-01-03 15:05:00+00:00,126.8500,126.895,126.4400,3763,126.505,362900,126.625878,0.002766,126.80786,127.06139,...,-2.719298,-40.134546,-4.465828,False,-9.402193e+06,-43.321203,-24.707011,0.967187,0.0,0
2023-01-03 15:06:00+00:00,126.5800,126.870,126.5300,2930,126.860,240388,126.664497,-0.002128,126.71786,126.94739,...,-2.551580,-47.104800,-4.057573,False,-9.798127e+06,-39.135081,-24.923630,0.907692,0.0,0
2023-01-03 15:07:00+00:00,126.7100,126.870,126.5000,3189,126.580,289721,126.666956,0.001027,126.69186,126.87129,...,-2.402012,-38.479113,-3.693347,False,-9.594539e+06,-32.305140,-24.862300,0.792548,0.0,0
2023-01-03 15:08:00+00:00,126.7066,126.760,126.5601,2861,126.705,255144,126.687038,-0.000027,126.66934,126.81495,...,-2.241145,-38.583764,-3.567436,False,-9.590454e+06,-32.682043,-24.646221,0.705603,0.0,0


In [7]:
# We should use the .shift() function so that our algorithm predicts the minute before realtime
# Drop the row with NaN values 
X = X.shift().dropna()

X.head()

,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,SMA10,...,COPP,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC,signal
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-01-03 15:05:00+00:00,126.5001,126.880,126.4166,5733.0,126.820,513722.0,126.624892,-0.002516,126.87386,127.15340,...,-2.878051,-65.583650,-4.458619,False,-9.952525e+06,-43.838967,-25.409400,1.034633,0.0,0.0
2023-01-03 15:06:00+00:00,126.8500,126.895,126.4400,3763.0,126.505,362900.0,126.625878,0.002766,126.80786,127.06139,...,-2.719298,-40.134546,-4.465828,False,-9.402193e+06,-43.321203,-24.707011,0.967187,0.0,0.0
2023-01-03 15:07:00+00:00,126.5800,126.870,126.5300,2930.0,126.860,240388.0,126.664497,-0.002128,126.71786,126.94739,...,-2.551580,-47.104800,-4.057573,False,-9.798127e+06,-39.135081,-24.923630,0.907692,0.0,0.0
2023-01-03 15:08:00+00:00,126.7100,126.870,126.5000,3189.0,126.580,289721.0,126.666956,0.001027,126.69186,126.87129,...,-2.402012,-38.479113,-3.693347,False,-9.594539e+06,-32.305140,-24.862300,0.792548,0.0,0.0
2023-01-03 15:09:00+00:00,126.7066,126.760,126.5601,2861.0,126.705,255144.0,126.687038,-0.000027,126.66934,126.81495,...,-2.241145,-38.583764,-3.567436,False,-9.590454e+06,-32.682043,-24.646221,0.705603,0.0,0.0


We may look to consider what it would do if we changed the amount that we shifted by. Perhaps we tried predicting 5 minutes into the future... how about an hour?

In [8]:
# y is the NEW signal column
y = df[("new_signal")]

y


timestamp
2023-01-03 15:04:00+00:00    1
2023-01-03 15:05:00+00:00    1
2023-01-03 15:06:00+00:00    1
2023-01-03 15:07:00+00:00    1
2023-01-03 15:08:00+00:00    1
                            ..
2024-01-17 00:47:00+00:00    1
2024-01-17 00:52:00+00:00    1
2024-01-17 00:55:00+00:00    1
2024-01-17 00:57:00+00:00    1
2024-01-17 00:58:00+00:00    1
Name: new_signal, Length: 153682, dtype: int64

In [9]:
# Set start of training period
training_begin = X.index.min()

print(training_begin)

2023-01-03 15:05:00+00:00


In [10]:
# Select ending period for the training data. Since we pulled a year's worth of data
# we will train on 9 months and then test with the rest
training_end = X.index.min() + DateOffset(months=9)

print(training_end)

2023-10-03 15:05:00+00:00


In [11]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

In [12]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
display(X_train_scaled.shape)
display(X_test_scaled.shape)

(113680, 98)

(40002, 98)

In [14]:
# from imblearn.over_sampling import RandomOverSampler
# # Use RandomOverSampler to resample the datase using random_state=1
# ros = RandomOverSampler(random_state=1)
# X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)

In [15]:
# display(X_train_scaled.shape)
# display(X_test_scaled.shape)

### Now that we have scaled our data, we can build our neural network.

In [16]:
num_predictors = len(X.columns)

# We have 2 possible outcomes, and we are trying to predict the stock/indicators to be in position -1 or 1
num_classes = 1

num_predictors

98

In [17]:
nn_model = Sequential()

In [18]:
# Add dense layer(s)
nn_model.add(Dense(10, input_dim=num_predictors, activation='relu'))

In [19]:
# Drop-out layer(s)
# nn_model.add(Dropout(.2,input_shape=(10,)))

# Add dense layer, add Regularization
#model.add(Dense(5, activation='relu', kernel_regularized=l2(0.01), bias_regularized=l2(0.01)))

In [20]:
# Add output layer
# Number of outputs equals number of classes
#nn_model.add(Dense(num_classes))
nn_model.add(Dense(num_classes, activation="sigmoid"))

In [21]:
# Compile model
nn_model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])

# Summarize model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                990       
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 1001 (3.91 KB)
Trainable params: 1001 (3.91 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
# Fit model
num_epochs = 100

nn_model.fit(X_train_scaled, y_train,
          epochs=num_epochs,
          batch_size=100,
          validation_split=0.2,     # This 'validation_split' is telling the neural network to keep 20% of the data to validate its score on the training set... this is to help AVOID OVERFITTING. 
          shuffle=True)

Epoch 1/100
910/910 [==============================] - 1s 592us/step - loss: -1.3059 - accuracy: 0.0567 - val_loss: -2.5598 - val_accuracy: 0.0882
Epoch 2/100
910/910 [==============================] - 1s 616us/step - loss: -11.0217 - accuracy: 0.0690 - val_loss: -9.5425 - val_accuracy: 0.1002
Epoch 3/100
910/910 [==============================] - 1s 820us/step - loss: -30.4580 - accuracy: 0.0722 - val_loss: -21.3833 - val_accuracy: 0.1052
Epoch 4/100
910/910 [==============================] - 0s 535us/step - loss: -57.2695 - accuracy: 0.0750 - val_loss: -35.9712 - val_accuracy: 0.1065
Epoch 5/100
910/910 [==============================] - 0s 527us/step - loss: -90.5252 - accuracy: 0.0739 - val_loss: -53.6904 - val_accuracy: 0.1075
Epoch 6/100
910/910 [==============================] - 0s 503us/step - loss: -129.8287 - accuracy: 0.0746 - val_loss: -73.6942 - val_accuracy: 0.1057
Epoch 7/100
910/910 [==============================] - 0s 536us/step - loss: -174.5690 - accuracy: 0.0747 - 

KeyboardInterrupt: 

In [ ]:
# Show model loss and accuracy

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Now we backtest with the TEST portion of the data

In [ ]:
# Predict values using testing data
nn_test_predictions = nn_model.predict(X_test_scaled)
nn_train_predictions = nn_model.predict(X_train_scaled)

In [ ]:
# # Evaluate the model using a classification report
# from sklearn.metrics import classification_report
# training_report = classification_report(nn_train_predictions, nn_test_predictions)
# print(training_report)

In [ ]:
# # Evaluate the model's ability to predict the trading signal for the testing data using a classification report
# training_report = classification_report(y_test, testing_signal_predictions)
# print(training_report)

In [ ]:
# # Save model history for further manipulation
# model_history = model.history.model_history.keys()

In [ ]:
# # Now we can plot the accuracy for training and validation

# training_results = pd.DataFrame(index=range(1, num_epochs+1))
# training_results['Training'] = model_history['categorical_accuracy']
# training_results['Validation'] = model_history['val_categorical_accuracy']
# training_results.plot(title = 'Training and Validation Performance')

### Now using the 3 minute

In [ ]:
# Import indicator dataframe
df = pd.read_csv("../algotrader2/resources/aapl_3min_pivot_point_indicator_df.csv")
df.head()

# df = pd.read_csv("../algotrader2/resources/aapl_1min_pivot_point_indicator_df.csv")
# df.head()

In [ ]:
# Create our timestamp column as a datetime index, then save it as our index
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)
# X is everything except the signal column
X = df.drop('new_signal', axis=1)
# We should use the .shift() function so that our algorithm predicts the minute before realtime
# Drop the row with NaN values 
X = X.shift().dropna()
display(X.head())
y = df[("new_signal")]
# Set start of training period
training_begin = X.index.min()
print(f"Start date: {training_begin}")
# Select ending period for the training data. Since we pulled a year's worth of data
# we will train on 9 months and then test with the rest
training_end = X.index.min() + DateOffset(months=9)
print(f"End date: {training_end}")
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

In [ ]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

display(X_train_scaled.shape)
display(X_test_scaled.shape)

# from imblearn.over_sampling import RandomOverSampler
# # Use RandomOverSampler to resample the datase using random_state=1
# ros = RandomOverSampler(random_state=1)
# X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)
# display(X_train_scaled.shape)
# display(X_test_scaled.shape)

In [ ]:
# NEURAL NETWORK
num_predictors = len(X.columns)
num_classes = 1
nn_model = Sequential()
# Add dense layer(s)
nn_model.add(Dense(10, input_dim=num_predictors, activation='relu'))
# Drop-out layer(s)
# nn_model.add(Dropout(.2,input_shape=(10,)))
# Add dense layer, add Regularization
#model.add(Dense(5, activation='relu', kernel_regularized=l2(0.01), bias_regularized=l2(0.01)))
# Add output layer
# Number of outputs equals number of classes
#nn_model.add(Dense(num_classes))
nn_model.add(Dense(num_classes, activation="sigmoid"))


In [ ]:
# Compile model
nn_model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])

# Summarize model
nn_model.summary()

In [ ]:
# Fit model
num_epochs = 100

nn_model.fit(X_train_scaled, y_train,
          epochs=num_epochs,
          batch_size=100,
          validation_split=0.2,     # This 'validation_split' is telling the neural network to keep 20% of the data to validate its score on the training set... this is to help AVOID OVERFITTING. 
          shuffle=True)

In [ ]:
# Show model loss and accuracy

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Predict values using testing data
nn_test_predictions = nn_model.predict(X_test_scaled)
nn_train_predictions = nn_model.predict(X_train_scaled)

In [ ]:
# # Evaluate the model using a classification report
# from sklearn.metrics import classification_report
# training_report = classification_report(nn_train_predictions, nn_test_predictions)
# print(training_report)

# # Evaluate the model's ability to predict the trading signal for the testing data using a classification report
# training_report = classification_report(y_test, testing_signal_predictions)
# print(training_report)

# # Save model history for further manipulation
# model_history = model.history.model_history.keys()

### Now 15 minute

In [ ]:
# Import indicator dataframe
df = pd.read_csv("../algotrader2/resources/aapl_15min_pivot_point_indicator_df.csv")
df.head()

In [ ]:
df.head(100)

In [ ]:
# Create our timestamp column as a datetime index, then save it as our index
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)
# X is everything except the signal column
X = df.drop('new_signal', axis=1)
# We should use the .shift() function so that our algorithm predicts the minute before realtime
# Drop the row with NaN values 
X = X.shift().dropna()
display(X.head())
y = df[("new_signal")]
# Set start of training period
training_begin = X.index.min()
print(f"Start date: {training_begin}")
# Select ending period for the training data. Since we pulled a year's worth of data
# we will train on 9 months and then test with the rest
training_end = X.index.min() + DateOffset(months=9)
print(f"End date: {training_end}")
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

In [ ]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

display(X_train_scaled.shape)
display(X_test_scaled.shape)

# # from imblearn.over_sampling import RandomOverSampler
# # Use RandomOverSampler to resample the datase using random_state=1
# ros = RandomOverSampler(random_state=1)
# X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)
# display(X_train_scaled.shape)
# display(X_test_scaled.shape)

In [ ]:
# NEURAL NETWORK
num_predictors = len(X.columns)
num_classes = 1
nn_model = Sequential()
# Add dense layer(s)
nn_model.add(Dense(10, input_dim=num_predictors, activation='relu'))
# Drop-out layer(s)
# nn_model.add(Dropout(.2,input_shape=(10,)))
# Add dense layer, add Regularization
#model.add(Dense(5, activation='relu', kernel_regularized=l2(0.01), bias_regularized=l2(0.01)))
# Add output layer
# Number of outputs equals number of classes
#nn_model.add(Dense(num_classes))
nn_model.add(Dense(num_classes, activation="sigmoid"))


In [ ]:
# Compile model
nn_model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])

# Summarize model
nn_model.summary()

In [ ]:
# Fit model
num_epochs = 100

nn_model.fit(X_train_scaled, y_train,
          epochs=num_epochs,
          batch_size=100,
          validation_split=0.2,     # This 'validation_split' is telling the neural network to keep 20% of the data to validate its score on the training set... this is to help AVOID OVERFITTING. 
          shuffle=True)

In [ ]:
# Show model loss and accuracy

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Predict values using testing data
nn_test_predictions = nn_model.predict(X_test_scaled)
nn_train_predictions = nn_model.predict(X_train_scaled)

In [ ]:
nn_test_predictions[:500]

In [ ]:
# # Evaluate the model using a classification report
# from sklearn.metrics import classification_report
# training_report = classification_report(nn_train_predictions, nn_test_predictions)
# print(training_report)

# # Evaluate the model's ability to predict the trading signal for the testing data using a classification report
# training_report = classification_report(y_test, testing_signal_predictions)
# print(training_report)

# # Save model history for further manipulation
# model_history = model.history.model_history.keys()

In [ ]:
# # Deployment (Hypothetical)
# while True:
#     current_data = your_trading_api.get_real_time_data()
#     current_data_processed = preprocess_data(current_data)
#     prediction = model.predict(current_data_processed)
#     if prediction > some_threshold:
#         your_trading_api.execute_trade()

# # Placeholder Functions
# def combine_data(historical, news):
#     # Combine and return data
#     pass

# def split_data(data):
#     # Split and return data
#     pass

# def backtest_strategy(model, data):
#     # Implement backtesting logic
#     pass

# def preprocess_data(data):
#     # Data preprocessing steps
#     pass